# Quantum Square Distance Classifier

Importing what is needed

In [1]:
import numpy as np

from sympy.physics.quantum.qapply import qapply
from sympy.physics.quantum.qubit import IntQubit, Qubit, measure_partial, matrix_to_qubit
from sympy.physics.quantum import TensorProduct
from sympy.physics.quantum.gate import HadamardGate, IdentityGate
from sympy.physics.quantum.represent import represent
from sympy import sqrt, simplify, preorder_traversal


We have two points

In [2]:
# price, room , survival
data = np.array([[0.921, 0.390, 1], [0.141, 0.990, 0]])
data

array([[0.921, 0.39 , 1.   ],
       [0.141, 0.99 , 0.   ]])

New passaenger we want to know the survival of

In [3]:
x_point = np.array([0.866, 0.500])
x_point

array([0.866, 0.5  ])

In [4]:
# calculate the distance of the new point
dists = []
for i in range(2):
    d = np.linalg.norm(x_point - data[i][0:2])
    dists.append(1. - 0.25 * d)

dists = dists / np.sum(dists)

print (dists)


[0.5537045 0.4462955]


In [5]:
# this is the quantum state where we have:
# Passenger 1 features
# Passenger 2 features
# Passenger 3 features   TWICE!
# Passenger 3 features

s_truth = 1/sqrt(4)*(0.921*Qubit(0, 0, 0, 1) +
                 0.390*Qubit(0, 0, 1, 1) +
                 0.141*Qubit(0, 1, 0, 0) +
                 0.99*Qubit(0, 1, 1, 0) +
                 0.866*Qubit(1, 0, 0, 1) +
                 0.500*Qubit(1, 0, 1, 1) +
                 0.866*Qubit(1, 1, 0, 0) +
                 0.500*Qubit(1, 1, 1, 0))
print(f"t={s_truth}")

data = np.array([[0.921, 0.390, 1], [0.141, 0.990, 0], [0.866, 0.500, 1], [0.866, 0.500, 0]])

s = 0

for i in range(4):
    for j in range(2):
        b = "{0:03b}".format(i*2+j)  # assigning a number between 0 and 7 to each 
        b += str(int(data[i][2])) # put one or zero if the passenger survived or not
        s += 1/sqrt(4)*Qubit(b)*data[i][j]

print(f"s={s}")
assert simplify(s - s_truth) == 0

t=(0.921*|0001> + 0.39*|0011> + 0.141*|0100> + 0.99*|0110> + 0.866*|1001> + 0.5*|1011> + 0.866*|1100> + 0.5*|1110>)/2
s=0.4605*|0001> + 0.195*|0011> + 0.0705*|0100> + 0.495*|0110> + 0.433*|1001> + 0.25*|1011> + 0.433*|1100> + 0.25*|1110>


In [6]:
# we apply Hadamard to the first qubit
g = IdentityGate(0)*IdentityGate(1)*IdentityGate(2)*HadamardGate(3)
x = qapply(g*s)
x

truth = 1/sqrt(8)*((0.921 + 0.866)*Qubit(0, 0, 0, 1) +
                   (0.390 + 0.500)*Qubit(0, 0, 1, 1) +
                   (0.141 + 0.866)*Qubit(0, 1, 0, 0) +
                   (0.990 + 0.500)*Qubit(0, 1, 1, 0) +
                   (0.921 - 0.866)*Qubit(1, 0, 0, 1) +
                   (0.390 - 0.500)*Qubit(1, 0, 1, 1) +
                   (0.141 - 0.866)*Qubit(1, 1, 0, 0) +
                   (0.990 - 0.500)*Qubit(1, 1, 1, 0))
print(truth)
assert simplify(x - truth) == 0

sqrt(2)*(1.787*|0001> + 0.89*|0011> + 1.007*|0100> + 1.49*|0110> + 0.055*|1001> - 0.11*|1011> - 0.725*|1100> + 0.49*|1110>)/4


In [7]:
# measure the first qubit
measure = measure_partial(x, (3,))
measure = measure[0] # get measure where first qubit is zero

p0 = 0
p1 = 0

for arg in preorder_traversal(measure):
    print (arg)
    if isinstance(arg, Qubit):
        print(arg.qubit_values)
        if arg.qubit_values[3] == 0:
            print ("YES")
        


# p(q4 == 1 )
p0 = 0.470275598868077*sqrt(2)*0.470275598868077*sqrt(2) + 0.234216722435696*sqrt(2)*0.234216722435696*sqrt(2)
print(p0)

# p(q4 == 0 )
p1 = 0.265007010666006*sqrt(2)*0.265007010666006*sqrt(2) + 0.392115636437288*sqrt(2)*0.392115636437288*sqrt(2)
print(p1)

# matching results we found classicaly
assert np.abs(p0 - dists[0]) < 1e-2
assert np.abs(p1 - dists[1]) < 1e-2


(0.470275598868077*sqrt(2)*|0001> + 0.234216722435696*sqrt(2)*|0011> + 0.265007010666006*sqrt(2)*|0100> + 0.392115636437288*sqrt(2)*|0110>, 0.902399459631612)
0.470275598868077*sqrt(2)*|0001> + 0.234216722435696*sqrt(2)*|0011> + 0.265007010666006*sqrt(2)*|0100> + 0.392115636437288*sqrt(2)*|0110>
0.392115636437288*sqrt(2)*|0110>
0.392115636437288
sqrt(2)
2
1/2
|0110>
(0, 1, 1, 0)
YES
0
1
1
0
0.234216722435696*sqrt(2)*|0011>
0.234216722435696
sqrt(2)
2
1/2
|0011>
(0, 0, 1, 1)
0
0
1
1
0.265007010666006*sqrt(2)*|0100>
0.265007010666006
sqrt(2)
2
1/2
|0100>
(0, 1, 0, 0)
YES
0
1
0
0
0.470275598868077*sqrt(2)*|0001>
0.470275598868077
sqrt(2)
2
1/2
|0001>
(0, 0, 0, 1)
0
0
0
1
0.902399459631612
0.552033223918497
0.447966776081504
